# Load Packages, Variables

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import time
import gc

import numpy as np
from google.cloud import bigquery
from google.cloud import storage
import gcsfs

import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)


from tensorflow import keras
from tensorflow.keras import layers
import seaborn as sns
from pandas.tseries.offsets import BDay

from tensorflow.keras.layers import Embedding
from tensorflow.keras import activations
from tensorflow.keras import backend as K
from tensorflow.keras import initializers
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error
from datetime import datetime
import matplotlib.pyplot as plt
import pickle5 as pickle


from ficc.utils.nelson_siegel_model import *
from ficc.utils.diff_in_days import *
from ficc.utils.auxiliary_functions import sqltodf


from IPython.display import display, HTML
import os


from ficc.data.process_data import process_data
from ficc.utils.auxiliary_variables import PREDICTORS, NON_CAT_FEATURES, BINARY, CATEGORICAL_FEATURES, IDENTIFIERS, PURPOSE_CLASS_DICT, NUM_OF_DAYS_IN_YEAR
from ficc.utils.gcp_storage_functions import upload_data, download_data
from ficc.utils.auxiliary_variables import RELATED_TRADE_BINARY_FEATURES, RELATED_TRADE_NON_CAT_FEATURES, RELATED_TRADE_CATEGORICAL_FEATURES

import sys
sys.path.append('../')
from ficc_keras_utils import *
import ficc_keras_utils
from lgbm_tools import *
from ficc_debiasing import *

pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(f'TF Version: {tf.__version__}')

2023-07-19 18:59:21.731161: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-19 18:59:21.743565: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-19 18:59:21.745205: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


Initializing pandarallel with 16.0 cores
INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
TF Version: 2.7.0


In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/jupyter/ficc/isaac_creds.json"
os.environ['TF_GPU_THREAD_MODE'] = 'gpu_private'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
pd.options.mode.chained_assignment = None

bq_client = bigquery.Client()
storage_client = storage.Client()

##COMMON VARIABLES
#DATA WINDOW
train_start = '2023-05-01'
train_end = '2023-06-01'
test_start = '2023-06-01'
test_end = '2023-07-01'
#MODEL PARAMETERS 
VALIDATION_SPLIT = ficc_keras_utils.VALIDATION_SPLIT
LEARNING_RATE = ficc_keras_utils.LEARNING_RATE
BATCH_SIZE = 1000 #ficc_keras_utils.BATCH_SIZE
NUM_EPOCHS = 75 #ficc_keras_utils.NUM_EPOCHS
DROPOUT = 0.01 #ficc_keras_utils.DROPOUT

##NOTEBOOK SPECIFIC VARIABLES 
TRADE_SEQUENCE_LENGTH = 2
NUM_FEATURES = 6
# target_variable = 'new_ys_diff'
# target_variable = 'new_ys' 
target_variable = 'new_ys' 

In [3]:
print(train_start ,
train_end ,
test_start ,
test_end)

2023-05-01 2023-06-01 2023-06-01 2023-07-01


# Load Data and Train NN

In [4]:
%time processed_data = load_data_from_pickle('processed_file_FULL_2023-07-12-17:22.pkl')

File available, loading pickle
CPU times: user 6.7 s, sys: 2.73 s, total: 9.43 s
Wall time: 9.43 s


In [5]:
processed_data = processed_data[~(processed_data.trade_history.apply(lambda x: x.shape) == (5,6))]

In [6]:
processed_data['trade_history'] = processed_data.trade_history.apply(lambda x: x[:,[0,2,3,4,5,6]])

In [7]:
processed_data['target_attention_features'] = processed_data['target_attention_features'].apply(lambda x: x[[0],:])

## Checking data

In [8]:
processed_data.trade_date.min(), processed_data.trade_date.max()

(Timestamp('2023-05-01 00:00:00'), Timestamp('2023-07-11 00:00:00'))

In [9]:
processed_data[['yield','new_real_time_ficc_ycl']].describe()

,yield,new_real_time_ficc_ycl
count,1746702.000,1746702.000
mean,364.498,323.013
std,91.917,41.183
min,0.100,263.663
25%,307.000,297.164
50%,350.300,310.806
75%,411.000,336.136
max,9892.500,541.456


In [10]:
processed_data.reset_index(drop=True, inplace=True)

In [11]:
processed_data['new_ys_realtime'] = processed_data['yield'] - processed_data['new_real_time_ficc_ycl']
processed_data['new_ys'] = processed_data['yield'] - processed_data['new_ficc_ycl']

In [12]:
##### EXTRA DATA PREPROCESSING #####
ttype_dict = { (0,0):'D', (0,1):'S', (1,0):'P' }

ys_variants = ["max_ys", "min_ys", "max_qty", "min_ago", "D_min_ago", "P_min_ago", "S_min_ago"]
ys_feats = ["_ys", "_ttypes", "_ago", "_qdiff"]
D_prev = dict()
P_prev = dict()
S_prev = dict()

def get_trade_history_columns():
    '''
    This function is used to create a list of columns
    '''
    YS_COLS = []
    for prefix in ys_variants:
        for suffix in ys_feats:
            YS_COLS.append(prefix + suffix)
    return YS_COLS

def extract_feature_from_trade(row, name, trade):
    yield_spread = trade[0]
    ttypes = ttype_dict[(trade[3],trade[4])] + row.trade_type
    seconds_ago = trade[5]
    quantity_diff = np.log10(1 + np.abs(10**trade[2] - 10**row.quantity))
    return [yield_spread, ttypes,  seconds_ago, quantity_diff]

def trade_history_derived_features(row):
    trade_history = row.trade_history
    trade = trade_history[0]
    
    D_min_ago_t = D_prev.get(row.cusip,trade)
    D_min_ago = 9        

    P_min_ago_t = P_prev.get(row.cusip,trade)
    P_min_ago = 9
    
    S_min_ago_t = S_prev.get(row.cusip,trade)
    S_min_ago = 9
    
    max_ys_t = trade; max_ys = trade[0]
    min_ys_t = trade; min_ys = trade[0]
    max_qty_t = trade; max_qty = trade[2]
    min_ago_t = trade; min_ago = trade[5]
    
    for trade in trade_history[0:]:
        #Checking if the first trade in the history is from the same block
        if trade[5] == 0: 
            continue
 
        if trade[0] > max_ys: 
            max_ys_t = trade
            max_ys = trade[0]
        elif trade[0] < min_ys: 
            min_ys_t = trade; 
            min_ys = trade[0]

        if trade[2] > max_qty: 
            max_qty_t = trade 
            max_qty = trade[2]
        if trade[5] < min_ago: 
            min_ago_t = trade; 
            min_ago = trade[5]
            
        side = ttype_dict[(trade[3],trade[4])]
        if side == "D":
            if trade[5] < D_min_ago: 
                D_min_ago_t = trade; D_min_ago = trade[5]
                D_prev[row.cusip] = trade
        elif side == "P":
            if trade[5] < P_min_ago: 
                P_min_ago_t = trade; P_min_ago = trade[5]
                P_prev[row.cusip] = trade
        elif side == "S":
            if trade[5] < S_min_ago: 
                S_min_ago_t = trade; S_min_ago = trade[5]
                S_prev[row.cusip] = trade
        else: 
            print("invalid side", trade)
    
    trade_history_dict = {"max_ys":max_ys_t,
                          "min_ys":min_ys_t,
                          "max_qty":max_qty_t,
                          "min_ago":min_ago_t,
                          "D_min_ago":D_min_ago_t,
                          "P_min_ago":P_min_ago_t,
                          "S_min_ago":S_min_ago_t}

    return_list = []
    for variant in ys_variants:
        feature_list = extract_feature_from_trade(row,variant,trade_history_dict[variant])
        return_list += feature_list
    
    return return_list

def process_data(data): 
    data['ted-rate'] = (data['t_rate_10'] - data['t_rate_2']) * 100
    
    data = data[(data.days_to_call == 0) | (data.days_to_call > np.log10(400))]
    data = data[(data.days_to_refund == 0) | (data.days_to_refund > np.log10(400))]
    data = data[(data.days_to_maturity == 0) | (data.days_to_maturity > np.log10(400))]
    data = data[data.days_to_maturity < np.log10(30000)]
    data['trade_history_sum'] = data.trade_history.parallel_apply(lambda x: np.sum(x))
    data.issue_amount = data.issue_amount.replace([np.inf, -np.inf], np.nan)
    data.dropna(inplace=True, subset=PREDICTORS+['trade_history_sum'])
    data.purpose_sub_class.fillna(0, inplace=True)
    
    return data

In [13]:
%%time
temp = processed_data[['cusip','trade_history','quantity','trade_type']].parallel_apply(trade_history_derived_features, axis=1)
YS_COLS = get_trade_history_columns()
processed_data[YS_COLS] = pd.DataFrame(temp.tolist(), index=processed_data.index)

CPU times: user 41.2 s, sys: 5.81 s, total: 47 s
Wall time: 1min 31s


In [14]:
%%time
processed_data['ttypes'] = (processed_data.last_trade_type.astype(str) + processed_data.trade_type.astype(str)).astype('category')
processed_data['diff_size'] = (processed_data.par_traded.astype(float) - processed_data.last_size).astype(np.float32)
processed_data['abs_last_yield_spread'] = np.abs(processed_data['last_yield_spread'])
processed_data['abs_diff_size'] = np.abs(processed_data['diff_size'])
processed_data['days_duration'] = (processed_data.last_calc_date - processed_data.last_settlement_date).dt.days

CPU times: user 853 ms, sys: 28 ms, total: 881 ms
Wall time: 880 ms


In [15]:
%%time
processed_data['trade_history_sum'] = processed_data.trade_history.parallel_apply(lambda x: np.sum(x))
processed_data = processed_data.dropna(subset=['trade_history_sum'])
processed_data.purpose_sub_class.fillna(0, inplace=True)

CPU times: user 12.5 s, sys: 3.62 s, total: 16.1 s
Wall time: 17.5 s


In [16]:
# for col in NON_CAT_FEATURES:
#     print(col + ':' + str(train_dataframe[col].isna().sum()))

In [17]:
for col in ['new_ficc_ycl', 'yield_spread']:
    if col in NON_CAT_FEATURES:
        NON_CAT_FEATURES.remove(col)
    if col in PREDICTORS:
        PREDICTORS.remove(col)

In [18]:
processed_data = process_data(processed_data)

In [19]:
additional_features = ['ttypes', 'diff_size', 'abs_last_yield_spread', 'abs_diff_size', 'days_duration']
YS_COLS = ['max_ys_ys', 'max_ys_ttypes', 'max_ys_ago', 'max_ys_qdiff', 'min_ys_ys', 'min_ys_ttypes', 'min_ys_ago', 'min_ys_qdiff', 'max_qty_ys', 'max_qty_ttypes', \
           'max_qty_ago', 'max_qty_qdiff', 'min_ago_ys', 'min_ago_ttypes', 'min_ago_ago', 'min_ago_qdiff', 'D_min_ago_ys', 'D_min_ago_ttypes', 'D_min_ago_ago', 'D_min_ago_qdiff',\
           'P_min_ago_ys', 'P_min_ago_ttypes', 'P_min_ago_ago', 'P_min_ago_qdiff', 'S_min_ago_ys', 'S_min_ago_ttypes', 'S_min_ago_ago', 'S_min_ago_qdiff']
    
for col in YS_COLS:
    if 'ttypes' in col and col not in PREDICTORS:
        PREDICTORS.append(col)
        CATEGORICAL_FEATURES.append(col)
    elif col not in PREDICTORS:
        NON_CAT_FEATURES.append(col)
        PREDICTORS.append(col)
        
if 'target_attention_features' not in PREDICTORS:
    PREDICTORS.append('target_attention_features')
    
if 'ficc_treasury_spread' not in PREDICTORS:
    PREDICTORS.append('ficc_treasury_spread')
    NON_CAT_FEATURES.append('ficc_treasury_spread')
    
for col in ['new_ficc_ycl', 'new_real_time_ficc_ycl']:     
    if col not in PREDICTORS:
        PREDICTORS.append(col)
        NON_CAT_FEATURES.append(col)
        
encoders = {}
fmax = {}
for f in CATEGORICAL_FEATURES:
    print(f)
    if f in ['rating', 'incorporated_state_code', 'trade_type', 'purpose_class']:
        fprep = preprocessing.LabelEncoder().fit(categorical_feature_values[f])
    else:
        fprep = preprocessing.LabelEncoder().fit(processed_data[f].drop_duplicates())
    fmax[f] = np.max(fprep.transform(fprep.classes_))
    encoders[f] = fprep
    
with open('encoders.pkl','wb') as file:
    pickle.dump(encoders,file)

rating
incorporated_state_code
trade_type
purpose_class
max_ys_ttypes
min_ys_ttypes
max_qty_ttypes
min_ago_ttypes
D_min_ago_ttypes
P_min_ago_ttypes
S_min_ago_ttypes


In [20]:
%%time

processed_data['trade_history_shortened'] = processed_data['trade_history'].apply(lambda x: x[:TRADE_SEQUENCE_LENGTH])

CPU times: user 810 ms, sys: 68.3 ms, total: 878 ms
Wall time: 877 ms


In [21]:
%%time

processed_data['trade_history_fixed'] = processed_data['trade_history_shortened'].apply(lambda x: x[::-1, :])

CPU times: user 1.89 s, sys: 95.4 ms, total: 1.99 s
Wall time: 2 s


In [22]:
processed_data.trade_date.min(), processed_data.trade_date.max(), len(processed_data)

(Timestamp('2023-05-01 00:00:00'), Timestamp('2023-07-11 00:00:00'), 1590785)

In [23]:
train_filter = (processed_data.trade_date < train_end) & (processed_data.trade_date >= train_start)
test_filter = (processed_data.trade_date >= test_start) & (processed_data.trade_date <test_end)
                                                            
train_dataframe = processed_data[train_filter]\
.sort_values(by='trade_date', ascending=True)\
.reset_index(drop=True)

test_dataframe = processed_data[test_filter]\
.sort_values(by='trade_date', ascending=True)\
.reset_index(drop=True)

print('Training data start: {}, end: {}'.format(train_dataframe.trade_date.min(),train_dataframe.trade_date.max()))
print('Test data start: {}, end: {}'.format(test_dataframe.trade_date.min(),test_dataframe.trade_date.max()))

Training data start: 2023-05-01 00:00:00, end: 2023-05-31 00:00:00
Test data start: 2023-06-01 00:00:00, end: 2023-06-30 00:00:00


In [24]:
print(train_dataframe.trade_date.min(), train_dataframe.trade_date.max(), len(train_dataframe))
print(test_dataframe.trade_date.min(), test_dataframe.trade_date.max(), len(test_dataframe))

2023-05-01 00:00:00 2023-05-31 00:00:00 705233
2023-06-01 00:00:00 2023-06-30 00:00:00 697202


In [25]:
for f in ['new_ficc_ycl', 'new_real_time_ficc_ycl']: 
    if f in NON_CAT_FEATURES:
        NON_CAT_FEATURES.remove(f)
    if f in BINARY:
        NON_CAT_FEATURES.remove(f)
    print(f'Removed {f}')

Removed new_ficc_ycl
Removed new_real_time_ficc_ycl


In [26]:
def create_input_new(df, trade_history_col, yield_history_cols):
    global encoders
    datalist = []
    
    for col in yield_history_cols:
        datalist.append(np.stack(df[col].to_numpy()))
        
    datalist.append(np.stack(df[trade_history_col].to_numpy()))
    datalist.append(np.stack(df['target_attention_features'].to_numpy()))

    noncat_and_binary = []
    for f in NON_CAT_FEATURES + BINARY:
        noncat_and_binary.append(np.expand_dims(df[f].to_numpy().astype('float32'), axis=1))
    datalist.append(np.concatenate(noncat_and_binary, axis=-1))
    
    for f in CATEGORICAL_FEATURES:
        encoded = encoders[f].transform(df[f])
        datalist.append(encoded.astype('float32'))
    
    return datalist

def generate_model_new(TRADE_SEQUENCE_LENGTH, yield_history_cols, yield_history_lengths, num_yield_history,
                      yield_history_normalizers, trade_history_normalizer, noncat_binary_normalizer):
    inputs = []
    layer = []

    ############## INPUT BLOCK ###################
    for i in range(num_yield_history):
        inputs.append(layers.Input(name=f"yield_history_input_{yield_history_cols[i]}", 
                                           shape=(yield_history_lengths[i], 1), 
                                           dtype = tf.float32))
    
    trade_history_input = layers.Input(name="trade_history_input", 
                                       shape=(TRADE_SEQUENCE_LENGTH, NUM_FEATURES), 
                                       dtype = tf.float32) 

    target_attention_input = layers.Input(name="target_attention_input", 
                                       shape=(1, 3), 
                                       dtype = tf.float32) 
    inputs.append(trade_history_input)
    inputs.append(target_attention_input)

    inputs.append(layers.Input(
        name="NON_CAT_AND_BINARY_FEATURES",
        shape=(len(NON_CAT_FEATURES + BINARY),)
    ))


    layer.append(noncat_binary_normalizer(inputs[num_yield_history+2]))
    ####################################################


    ############## TRADE HISTORY MODEL #################

    lstm_layer = layers.Bidirectional(layers.LSTM(50, 
                             activation='tanh',
                             input_shape=(TRADE_SEQUENCE_LENGTH,NUM_FEATURES),
                             return_sequences = True,
                             name='LSTM'))

    lstm_layer_2 = layers.Bidirectional(layers.LSTM(100, 
                                                    activation='tanh',
                                                    input_shape=(TRADE_SEQUENCE_LENGTH, 50),
                                                    return_sequences = True,
                                                    name='LSTM_2'))



    features = lstm_layer(trade_history_normalizer(inputs[num_yield_history]))
    features = lstm_layer_2(features)  
    
    
    attention_sequence = layers.Dense(200, activation='relu', name='attention_dense')(target_attention_input)
    attention = layers.Dot(axes=[2, 2])([features, attention_sequence])
    attention = layers.Activation('softmax')(attention)

    context_vector = layers.Dot(axes=[1, 1])([features, attention])
    context_vector = layers.Flatten(name='context_vector_flatten')(context_vector)


    trade_history_output = layers.Dense(100, 
                                        activation='relu')(context_vector)

    ####################################################
    
    ############## YIELD HISTORY MODEL #################
    yield_history_outputs = []
    for i in range(num_yield_history):
        yield_lstm_layer = layers.Bidirectional(layers.LSTM(50, 
                                 activation='tanh',
                                 input_shape=(yield_history_lengths[i], 1),
                                 return_sequences = False,
                                 name=f'Yield_History_LSTM_{yield_history_cols[i]}'))

        yield_features = yield_lstm_layer(yield_history_normalizers[i](inputs[i]))
        yield_history_outputs.append(layers.Dense(25, activation='relu')(yield_features))
 
    ####################################################

    ############## REFERENCE DATA MODEL ################
    global encoders
    for f in CATEGORICAL_FEATURES:
        fin = layers.Input(shape=(1,), name = f)
        inputs.append(fin)
        embedded = layers.Flatten(name = f + "_flat")( layers.Embedding(input_dim = fmax[f]+1,
                                                                        output_dim = max(30,int(np.sqrt(fmax[f]))),
                                                                        input_length= 1,
                                                                        name = f + "_embed")(fin))
        layer.append(embedded)


    reference_hidden = layers.Dense(400,
                                    activation='relu',
                                    name='reference_hidden_1')(layers.concatenate(layer, axis=-1))

    reference_hidden = layers.BatchNormalization()(reference_hidden)
    reference_hidden = layers.Dropout(DROPOUT)(reference_hidden)

    reference_hidden2 = layers.Dense(200,activation='relu',name='reference_hidden_2')(reference_hidden)
    reference_hidden2 = layers.BatchNormalization()(reference_hidden2)
    reference_hidden2 = layers.Dropout(DROPOUT)(reference_hidden2)

    reference_output = layers.Dense(100,activation='tanh',name='reference_hidden_3')(reference_hidden2)

    ####################################################

    feed_forward_input = layers.concatenate(yield_history_outputs+[reference_output, trade_history_output])

    hidden = layers.Dense(300,activation='relu')(feed_forward_input)
    hidden = layers.BatchNormalization()(hidden)
    hidden = layers.Dropout(DROPOUT)(hidden)

    hidden2 = layers.Dense(100,activation='relu')(hidden)
    hidden2 = layers.BatchNormalization()(hidden2)
    hidden2 = layers.Dropout(DROPOUT)(hidden2)
    final = layers.Dense(1)(hidden2)

    model = keras.Model(inputs=inputs, outputs=final)
    
    return model


def create_data_set_and_model(train_dataframe, test_dataframe, trade_history_col, yield_history_cols):
    
    if not isinstance(trade_history_col, str):
        raise ValueError('trade_history_col must be a string')
    
    if isinstance(yield_history_cols, str):
        num_yield_history = 1
        yield_history_cols = [yield_history_cols]
    else:
        num_yield_history = len(yield_history_cols)
    
    yield_history_lengths = [train_dataframe[x][0].shape[0] for x in yield_history_cols]
    
    TRADE_SEQUENCE_LENGTH = train_dataframe[trade_history_col][0].shape[0] 
    
    params = {'TRADE_SEQUENCE_LENGTH':TRADE_SEQUENCE_LENGTH, 
           'yield_history_cols':yield_history_cols, 
           'yield_history_lengths':yield_history_lengths, 
           'num_yield_history':num_yield_history }
    
    np.random.seed(1)
    val_idx = np.random.choice(range(len(train_dataframe)), 
                     size = int(VALIDATION_SPLIT*len(train_dataframe)),
                     replace=False)

    print(f'TRAINING DATA: N = {len(train_dataframe)-len(val_idx)}, MIN DATE = {train_dataframe.drop(val_idx, axis=0).trade_date.min()}, MAX DATE = {train_dataframe.drop(val_idx, axis=0).trade_date.max()}')
    print(f'VALIDATION DATA: N = {len(val_idx)}, MIN DATE = {train_dataframe.iloc[val_idx].trade_date.min()}, MAX DATE = {train_dataframe.iloc[val_idx].trade_date.max()}')
    print(f'TEST DATA: N = {len(test_dataframe)}, MIN DATE = {test_dataframe.trade_date.min()}, MAX DATE = {test_dataframe.trade_date.max()}')

    x_train = create_input_new(train_dataframe.drop(val_idx, axis=0), trade_history_col, yield_history_cols)
    y_train = train_dataframe.drop(val_idx, axis=0)[target_variable]

    x_val = create_input_new(train_dataframe.iloc[val_idx], trade_history_col, yield_history_cols)
    y_val = train_dataframe.iloc[val_idx][target_variable]

    x_test = create_input_new(test_dataframe, trade_history_col, yield_history_cols)
    y_test = test_dataframe[target_variable]    
    
    # Normalization layer for the yield history
    yield_history_normalizers = []
    for i in range(num_yield_history):
        yield_history_normalizers.append(Normalization(name=f'Yield_history_normalizer_{yield_history_cols[i]}'))
        yield_history_normalizers[i].adapt(x_train[i],batch_size=BATCH_SIZE)

    # Normalization layer for the trade history
    trade_history_normalizer = Normalization(name='Trade_history_normalizer')
    trade_history_normalizer.adapt(x_train[num_yield_history],batch_size=BATCH_SIZE)

    # Normalization layer for the non-categorical and binary features
    noncat_binary_normalizer = Normalization(name='Numerical_binary_normalizer')
    noncat_binary_normalizer.adapt(x_train[num_yield_history+2], batch_size = BATCH_SIZE)

    normalizers = {'yield_history_normalizers': yield_history_normalizers,
                  'trade_history_normalizer': trade_history_normalizer,
                  'noncat_binary_normalizer': noncat_binary_normalizer}

    return  params, normalizers, x_train, y_train, x_val, y_val, x_test, y_test, val_idx

In [27]:
def create_tf_data(x_train, y_train, shuffle=False, shuffle_buffer=1):
                     
    X=()
    for x in x_train:
        X += (tf.data.Dataset.from_tensor_slices(x),)
        

    temp = tf.data.Dataset.zip((X))
    del X
    dataset = tf.data.Dataset.zip((temp,
                        tf.data.Dataset.from_tensor_slices(y_train)))
    del temp
    if shuffle:
        shuffle_buffer = int(len(x_train[0])*shuffle_buffer)
        dataset = dataset.shuffle(shuffle_buffer)
        
    return dataset

In [28]:
def train_model_new(params, normalizers, x_train, y_train, x_val, y_val, shuffle, shuffle_buffer=1):
    tf.keras.backend.clear_session()
    gc.collect()

    TRADE_SEQUENCE_LENGTH = params.get('TRADE_SEQUENCE_LENGTH')
    yield_history_cols = params.get('yield_history_cols')
    yield_history_lengths = params.get('yield_history_lengths')
    num_yield_history = params.get('num_yield_history')
      
    yield_history_normalizers = normalizers.get('yield_history_normalizers')
    trade_history_normalizer = normalizers.get('trade_history_normalizer')
    noncat_binary_normalizer = normalizers.get('noncat_binary_normalizer')
       
    tf.keras.utils.set_random_seed(10)
    model = generate_model_new(TRADE_SEQUENCE_LENGTH, yield_history_cols, yield_history_lengths, num_yield_history, 
                               yield_history_normalizers, trade_history_normalizer, noncat_binary_normalizer)
    
    timestamp = datetime.now().strftime('%Y-%m-%d %H-%M')
    
    fit_callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=10,
        verbose=0,
        mode="auto",
        restore_best_weights=True),
        # time_callback,
        CSVLoggerTimeHistory(timestamp+'_training_logs_yield_history.csv', separator=",", append=False)]
    
    with tf.device('/cpu:0'):
        train_ds = create_tf_data(x_train, y_train, shuffle, shuffle_buffer)
        train_ds = train_ds.batch(BATCH_SIZE).prefetch(2).cache()
        val_ds = create_tf_data(x_val, y_val, shuffle = False)
        val_ds = val_ds.batch(BATCH_SIZE).prefetch(2).cache()
    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
          loss=keras.losses.MeanAbsoluteError())

    history= model.fit(train_ds,
                                      validation_data=val_ds,
                                        epochs=NUM_EPOCHS,     
                                        verbose=1, 
                                        callbacks=fit_callbacks,
                                        use_multiprocessing=True,
                                        workers=8)
    
    return history, model



In [29]:
for col in ['new_ficc_ycl', 'new_real_time_ficc_ycl']:     
    if col in PREDICTORS:
        PREDICTORS.remove(col)
    if col in NON_CAT_FEATURES:
        NON_CAT_FEATURES.remove(col)

In [30]:
trade_history_col = 'trade_history_shortened'
yield_history_cols = []

In [31]:
test_dataframe_sub = test_dataframe[test_dataframe.trade_date <= '2023-06-09']

In [32]:
# params, normalizers, x_train, y_train, x_val, y_val, x_test, y_test, val_idx = create_data_set_and_model(train_dataframe, test_dataframe, trade_history_col, yield_history_cols)
params, normalizers, x_train, y_train, x_val, y_val, x_test, y_test, val_idx = create_data_set_and_model(train_dataframe, test_dataframe[test_dataframe.trade_date <= '2023-06-09'], trade_history_col, yield_history_cols)

TRAINING DATA: N = 634710, MIN DATE = 2023-05-01 00:00:00, MAX DATE = 2023-05-31 00:00:00
VALIDATION DATA: N = 70523, MIN DATE = 2023-05-01 00:00:00, MAX DATE = 2023-05-31 00:00:00
TEST DATA: N = 235624, MIN DATE = 2023-06-01 00:00:00, MAX DATE = 2023-06-09 00:00:00


2023-07-19 19:02:23.169227: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-19 19:02:23.173126: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-19 19:02:23.175071: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-19 19:02:23.176676: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [33]:
print(len(x_train))
for i in range(len(x_train)):
    print(x_train[i].shape)

14
(634710, 2, 6)
(634710, 1, 3)
(634710, 48)
(634710,)
(634710,)
(634710,)
(634710,)
(634710,)
(634710,)
(634710,)
(634710,)
(634710,)
(634710,)
(634710,)


In [55]:
results = []
predictions = []
NUM_EPOCHS  = 75

for i in range(8):
    tf.keras.utils.set_random_seed(10)
    history, model = train_model_new(params, normalizers, x_train, y_train, x_val, y_val, True, shuffle_buffer = .75)
    pred = model.predict(x_test)
    predictions.append(pred)
    print('='*25+f' TRIAL {i}, MAE: {mean_absolute_error(pred,y_test)} '+'='*25)
    results.append([history, model])

Epoch 1/75


2023-07-18 22:22:19.572724: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:689] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "Tesla T4" frequency: 1590 num_cores: 40 environment { key: "architecture" value: "7.5" } environment { key: "cuda" value: "11020" } environment { key: "cudnn" value: "8100" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 4194304 shared_memory_size_per_multiprocessor: 65536 memory_size: 13007060992 bandwidth: 320064000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


635/635 [==============================] - ETA: 0s - loss: 43.3011

2023-07-18 22:22:44.384006: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:689] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "Tesla T4" frequency: 1590 num_cores: 40 environment { key: "architecture" value: "7.5" } environment { key: "cuda" value: "11020" } environment { key: "cudnn" value: "8100" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 4194304 shared_memory_size_per_multiprocessor: 65536 memory_size: 13007060992 bandwidth: 320064000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


635/635 [==============================] - 33s 27ms/step - loss: 43.3011 - val_loss: 39.0171
Epoch 2/75
635/635 [==============================] - 12s 18ms/step - loss: 39.3124 - val_loss: 36.5678
Epoch 3/75
635/635 [==============================] - 23s 36ms/step - loss: 35.7395 - val_loss: 34.0807
Epoch 4/75
635/635 [==============================] - 12s 18ms/step - loss: 31.3821 - val_loss: 30.0747
Epoch 5/75
635/635 [==============================] - 12s 18ms/step - loss: 26.0626 - val_loss: 25.0375
Epoch 6/75
635/635 [==============================] - 12s 18ms/step - loss: 20.3064 - val_loss: 17.4329
Epoch 7/75
635/635 [==============================] - 23s 36ms/step - loss: 15.3155 - val_loss: 13.0115
Epoch 8/75
635/635 [==============================] - 12s 18ms/step - loss: 12.1222 - val_loss: 10.9199
Epoch 9/75
635/635 [==============================] - 12s 18ms/step - loss: 10.7983 - val_loss: 10.1960
Epoch 10/75
635/635 [==============================] - 12s 19ms/step - loss

...

635/635 [==============================] - ETA: 0s - loss: 43.3010

2023-07-19 00:39:39.475487: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:689] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "Tesla T4" frequency: 1590 num_cores: 40 environment { key: "architecture" value: "7.5" } environment { key: "cuda" value: "11020" } environment { key: "cudnn" value: "8100" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 4194304 shared_memory_size_per_multiprocessor: 65536 memory_size: 13007060992 bandwidth: 320064000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


635/635 [==============================] - 38s 25ms/step - loss: 43.3010 - val_loss: 39.0396
Epoch 2/75
635/635 [==============================] - 10s 15ms/step - loss: 39.3128 - val_loss: 36.5533
Epoch 3/75
635/635 [==============================] - 10s 15ms/step - loss: 35.7387 - val_loss: 34.0406
Epoch 4/75
635/635 [==============================] - 21s 33ms/step - loss: 31.3824 - val_loss: 30.2720
Epoch 5/75
635/635 [==============================] - 10s 15ms/step - loss: 26.0625 - val_loss: 24.5545
Epoch 6/75
635/635 [==============================] - 10s 15ms/step - loss: 20.3047 - val_loss: 17.7263
Epoch 7/75
635/635 [==============================] - 9s 15ms/step - loss: 15.3179 - val_loss: 13.1001
Epoch 8/75
635/635 [==============================] - 10s 15ms/step - loss: 12.1198 - val_loss: 10.9011
Epoch 9/75
635/635 [==============================] - 21s 33ms/step - loss: 10.7975 - val_loss: 10.2138
Epoch 10/75
635/635 [==============================] - 10s 15ms/step - loss:

2023-07-19 00:54:41.170997: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:689] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "Tesla T4" frequency: 1590 num_cores: 40 environment { key: "architecture" value: "7.5" } environment { key: "cuda" value: "11020" } environment { key: "cudnn" value: "8100" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 4194304 shared_memory_size_per_multiprocessor: 65536 memory_size: 13007060992 bandwidth: 320064000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


========================= TRIAL 7, MAE: 9.74230706731968 =========================


In [56]:
#SECOND RUN ON 07/14; 100 EPOCHS
t = 0
for i, prediction in enumerate(predictions):
    print(f'Trial {i+1}: {mean_absolute_error(prediction, y_test):.3f}')
    t += mean_absolute_error(prediction, y_test)
print(f'Average MAE: {t/len(predictions):.3f}')

Trial 1: 9.774
Trial 2: 9.843
Trial 3: 9.851
Trial 4: 9.765
Trial 5: 9.820
Trial 6: 9.817
Trial 7: 9.853
Trial 8: 9.742
Average MAE: 9.808


In [40]:
model.save('baseline_model')

2023-07-18 18:27:51.158205: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: baseline_model/assets


INFO:tensorflow:Assets written to: baseline_model/assets


# Pricing Hypothetical Trades

In [35]:
def get_days(row, col):
    if pd.isna(row[col]) or pd.isna(row['settlement_date']):
        return 0
    else: 
        diff = diff_in_days_two_dates(row[col], row.settlement_date)
        if diff <= 0:
            return -1
        else:
            return diff
        
        
def sample_dataframe(df, N):
    
    group_name = df.name
    def index_to_dict(index_row):
        is_callable = index_row[0]
        interval = str(index_row[1].left)+'-'+str(index_row[1].right)
        rating = index_row[2]
        return {'is_callable':is_callable, 'interval': interval, 'rating':rating}
    
    df = df.drop_duplicates(subset='cusip')
    
    if len(df) < N:
        N = len(df)
    
    np.random.seed(1)
    result = df.sample(N, replace=False) 
    group_id = next(COUNT)
    result['group'] = group_id
    groupby_id_dict[group_id] = index_to_dict(group_name)
    return result


def create_synthetic_samples(df, col, vals):
    for cusip in df.cusip.unique():
        for val in vals:
            temp = df[df.cusip == cusip].iloc[0] 
            if val != temp[col]:
                temp[col] = val
                df = df.append(temp)
    return df

def make_summary(col):
    # col = 'is_callable'

    summary_df = synthetic_sampled_data.groupby(['cusip'])\
    ['predictions']\
    .agg(['std', max_min_f])\
    .rename({'<lambda_0>':'Max-Min'}, axis=1)\
    
    summary_df = summary_df.join(synthetic_sampled_data.set_index('cusip')[col])
    
    summary_df = summary_df.groupby(col).mean()
    # summary_df.columns = pd.MultiIndex.from_tuples(summary_df.columns)
    
    display(summary_df)
    
    return summary_df

In [36]:
data = test_dataframe[test_dataframe.trade_date <= '2023-06-09'].copy()
# data['days_to_call'] = data[['settlement_date','next_call_date']].apply(lambda x: get_days(x, 'next_call_date'), axis = 1)/NUM_OF_DAYS_IN_YEAR
data['maturity'] = data[['settlement_date','maturity_date']].apply(lambda x: get_days(x, 'maturity_date'), axis = 1)/NUM_OF_DAYS_IN_YEAR
data['maturity_bucket'] = pd.cut(data['maturity'], [0, 5, 10, 15, 20, 30, data.maturity.max()])

In [37]:
#We just look at 5 rating values for efficiency 
RATINGS = ['AAA',  'BBB', 'CCC', 'NR']
groupby_cols = ['is_callable','maturity_bucket','rating']

sampled_data = data[data.rating.isin(RATINGS)].groupby(groupby_cols)
COUNT = iter(range(len(sampled_data.groups)))
groupby_id_dict = dict()

#Sample 20 random cusips from each maturity bucket, rating and callable combination 
sampled_data = sampled_data.apply(lambda x: sample_dataframe(x, 20)).reset_index(drop=True)

%time synthetic_sampled_data = create_synthetic_samples(sampled_data, 'rating', RATINGS)

CPU times: user 1min 16s, sys: 73.4 ms, total: 1min 16s
Wall time: 1min 16s


In [38]:
sampled_data

,rtrs_control_number,cusip,yield,is_callable,refund_date,accrual_date,dated_date,next_sink_date,coupon,delivery_date,...,abs_last_yield_spread,abs_diff_size,days_duration,trade_history_sum,ted-rate,trade_history_shortened,trade_history_fixed,maturity,maturity_bucket,group
0,2023060106012200,13067WRB0,506.300,False,NaT,2020-08-06,2020-08-06,NaT,0.560,2020-08-06,...,196.191,60000.000,549,1339.388,-72.000,"[[196.19109531370867, 99.8, 4.602059841156006,...","[[188.88488489691935, 87.3, 5.0, 1.0, 0.0, 5.7...",1.489,"(0.0, 5.0]",0
1,2023060208551800,269696KP4,305.000,False,NaT,2018-12-20,2018-12-01,NaT,5.000,2018-12-20,...,1.941,5000.000,1542,-216.378,-81.000,"[[-1.9410871431837222, -59.0, 4.17609119415283...","[[-1.9410871431837222, -59.0, 4.17609119415283...",4.192,"(0.0, 5.0]",0
2,2023060108789700,161045QG7,289.200,False,NaT,2020-09-24,2020-09-24,NaT,5.000,2020-09-24,...,16.132,10000.000,1497,-381.400,-72.000,"[[-16.132136149069538, -72.6, 4.0, 0.0, 1.0, 5...","[[4.6833993241039025, -64.8, 4.0, 0.0, 0.0, 5....",4.072,"(0.0, 5.0]",0
3,2023060103514200,655867TX7,305.600,False,NaT,2016-10-19,2016-10-19,NaT,5.000,2016-10-19,...,13.485,15000.000,1220,-487.650,-72.000,"[[13.484656000666575, -84.1, 4.0, 1.0, 0.0, 5....","[[10.384656000666553, -87.2, 4.0, 0.0, 0.0, 5....",3.322,"(0.0, 5.0]",0
4,2023060509494600,92818MZH6,284.600,False,NaT,2018-08-06,2018-08-06,NaT,5.000,2018-08-06,...,4.739,0.000,1624,-528.767,-77.000,"[[-4.739394888894822, -89.0, 4.397940158843994...","[[-3.163124109371296, -125.1, 4.39794015884399...",4.400,"(0.0, 5.0]",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2023060105719200,65830RCV9,425.000,True,NaT,2019-12-17,2019-12-17,2050-01-01,4.000,2019-12-17,...,19.986,5000.000,11536,117.164,-72.000,"[[-19.985699461609443, 41.0, 4.301030158996582...","[[-18.485699461609386, 42.5, 4.301030158996582...",31.572,"(30.0, 46.136]",32
496,2023060702745900,888805CJ5,524.900,True,NaT,2021-04-13,2021-04-13,2028-06-01,0.000,2021-04-13,...,62.154,3000000.000,15335,1064.315,-77.000,"[[62.15424012169029, 135.8, 6.698969841003418,...","[[64.35424012169022, 138.0, 7.176091194152832,...",41.978,"(30.0, 46.136]",33
497,2023060101614700,13016NFC3,568.000,True,NaT,2020-06-10,2020-06-10,2027-06-01,0.000,2020-06-10,...,111.783,12055000.000,11778,1866.770,-72.000,"[[111.78281923384151, 175.0, 7.431363582611084...","[[109.78281923384151, 173.0, 7.431363582611084...",31.989,"(30.0, 46.136]",33
498,2023060103450000,118217CZ9,546.000,True,NaT,2020-03-04,2020-03-04,2024-06-01,5.000,2020-03-04,...,103.148,1500000.000,11684,1296.770,-72.000,"[[103.1482038867872, 162.0, 6.397940158843994,...","[[105.1482038867872, 164.0, 6.3324384689331055...",31.989,"(30.0, 46.136]",33


In [39]:
synthetic_sampled_data

,rtrs_control_number,cusip,yield,is_callable,refund_date,accrual_date,dated_date,next_sink_date,coupon,delivery_date,...,abs_last_yield_spread,abs_diff_size,days_duration,trade_history_sum,ted-rate,trade_history_shortened,trade_history_fixed,maturity,maturity_bucket,group
0,2023060106012200,13067WRB0,506.300,False,NaT,2020-08-06,2020-08-06,NaT,0.560,2020-08-06,...,196.191,60000.000,549,1339.388,-72.000,"[[196.19109531370867, 99.8, 4.602059841156006,...","[[188.88488489691935, 87.3, 5.0, 1.0, 0.0, 5.7...",1.489,"(0.0, 5.0]",0
1,2023060208551800,269696KP4,305.000,False,NaT,2018-12-20,2018-12-01,NaT,5.000,2018-12-20,...,1.941,5000.000,1542,-216.378,-81.000,"[[-1.9410871431837222, -59.0, 4.17609119415283...","[[-1.9410871431837222, -59.0, 4.17609119415283...",4.192,"(0.0, 5.0]",0
2,2023060108789700,161045QG7,289.200,False,NaT,2020-09-24,2020-09-24,NaT,5.000,2020-09-24,...,16.132,10000.000,1497,-381.400,-72.000,"[[-16.132136149069538, -72.6, 4.0, 0.0, 1.0, 5...","[[4.6833993241039025, -64.8, 4.0, 0.0, 0.0, 5....",4.072,"(0.0, 5.0]",0
3,2023060103514200,655867TX7,305.600,False,NaT,2016-10-19,2016-10-19,NaT,5.000,2016-10-19,...,13.485,15000.000,1220,-487.650,-72.000,"[[13.484656000666575, -84.1, 4.0, 1.0, 0.0, 5....","[[10.384656000666553, -87.2, 4.0, 0.0, 0.0, 5....",3.322,"(0.0, 5.0]",0
4,2023060509494600,92818MZH6,284.600,False,NaT,2018-08-06,2018-08-06,NaT,5.000,2018-08-06,...,4.739,0.000,1624,-528.767,-77.000,"[[-4.739394888894822, -89.0, 4.397940158843994...","[[-3.163124109371296, -125.1, 4.39794015884399...",4.400,"(0.0, 5.0]",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,2023060103450000,118217CZ9,546.000,True,NaT,2020-03-04,2020-03-04,2024-06-01,5.000,2020-03-04,...,103.148,1500000.000,11684,1296.770,-72.000,"[[103.1482038867872, 162.0, 6.397940158843994,...","[[105.1482038867872, 164.0, 6.3324384689331055...",31.989,"(30.0, 46.136]",33
498,2023060103450000,118217CZ9,546.000,True,NaT,2020-03-04,2020-03-04,2024-06-01,5.000,2020-03-04,...,103.148,1500000.000,11684,1296.770,-72.000,"[[103.1482038867872, 162.0, 6.397940158843994,...","[[105.1482038867872, 164.0, 6.3324384689331055...",31.989,"(30.0, 46.136]",33
499,2023060101076500,38122NB84,534.000,True,NaT,2021-12-15,2021-12-15,2029-06-01,0.000,2021-12-15,...,65.417,2000000.000,15702,1151.008,-72.000,"[[65.41674418743281, 150.0, 6.84509801864624, ...","[[65.41674418743281, 150.0, 7.0, 0.0, 1.0, 3.4...",42.989,"(30.0, 46.136]",33
499,2023060101076500,38122NB84,534.000,True,NaT,2021-12-15,2021-12-15,2029-06-01,0.000,2021-12-15,...,65.417,2000000.000,15702,1151.008,-72.000,"[[65.41674418743281, 150.0, 6.84509801864624, ...","[[65.41674418743281, 150.0, 7.0, 0.0, 1.0, 3.4...",42.989,"(30.0, 46.136]",33


In [60]:
%time synthetic_sampled_data = create_synthetic_samples(sampled_data)

CPU times: user 1min 37s, sys: 0 ns, total: 1min 37s
Wall time: 1min 37s


In [61]:
X_sample = create_input_new(sampled_data, trade_history_col, yield_history_cols)
X_sample_hypothetical_trades = create_input_new(synthetic_sampled_data, trade_history_col, yield_history_cols)

In [62]:
sampled_data_predictions  = model.predict(X_sample).flatten()
sampled_data_hypothetical_predictions  = model.predict(X_sample_hypothetical_trades).flatten()

In [63]:
sampled_data['predictions'] = sampled_data_predictions
synthetic_sampled_data['predictions'] = sampled_data_hypothetical_predictions

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  


In [64]:
mean_absolute_error(sampled_data['predictions'], sampled_data['new_ys']), mean_absolute_error(synthetic_sampled_data['predictions'], synthetic_sampled_data['new_ys'])

(13.70137205875082, 15.0045147946588)

In [65]:
max_min_f = lambda x: x.max() - x.min()

synthetic_sampled_data['original_rating'] = synthetic_sampled_data['group'].apply(lambda x: groupby_id_dict[x]['rating'])

make_summary('is_callable');
make_summary('original_rating');
make_summary('maturity_bucket');

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  This is separate from the ipykernel package so we can avoid doing imports until


,std,Max-Min
is_callable,,
False,5.074,12.128
True,4.526,10.882


,std,Max-Min
original_rating,,
AAA,4.193,9.505
BBB,5.355,12.805
CCC,9.848,21.623
MR,4.842,12.156
NR,4.841,11.457


,std,Max-Min
maturity_bucket,,
"(0.0, 5.0]",7.176,17.364
"(5.0, 10.0]",5.332,12.661
"(10.0, 15.0]",4.369,10.375
"(15.0, 20.0]",4.153,10.033
"(20.0, 30.0]",3.586,8.566
"(30.0, 46.136]",2.574,6.213


In [69]:
stouch .gitignoreynthetic_sampled_data.to_pickle('synthetic_sampled_data_baseline.pkl')